## Find the start position of a list of kmers(or regex) in a list of genomes includingon the reverse strand.

In [1]:
import pickle
import glob
import re
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
from Bio import SeqIO
from collections import defaultdict
from matplotlib import pyplot as plt 
%matplotlib inline

In [2]:
fasta_path = '/home4/youn01f/Desktop/workspace/amr/data/'
features_file = 'features.pkl'

### Read sequences from files into a dictionary
Key = strain
Value = sequence as a string

In [3]:
def get_sequences(filenames):
    sequences = defaultdict(str)
    for filename in filenames:
        strain = Path(filename).stem
        seq = '' 
        with open(filename, 'rt') as handle:
            for record in SeqIO.parse(handle, "fasta"):                   
                seq += str(record.seq)
        sequences[strain] = seq
        #print (strain, len(seq))       
    return (sequences)

In [4]:
filenames = glob.glob(f'{fasta_path}*.fna')
seqs = get_sequences(filenames)

Use the feature matrix to select some kmers that we should definitely find

In [5]:
features_df = pd.read_pickle(features_file)
X =features_df.to_numpy()
Z = np.count_nonzero(X, axis=0)
ind = np.where(Z==200) # select kmers that are in  200 of the sequences ( random choice)
kmers_df = features_df.iloc[:, ind[0]]
kmers = kmers_df.columns

In [6]:
def compliment(dna):
    """Return the reverse compliment of a sequence """
    bp = { 'A':'T', 'T':'A', 'C':'G','G':'C'}
    revdna =''    
    for i in range(len(dna)-1,-1,-1):
         revdna += bp[dna[i]]
    return (revdna)

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

### Using string function in find_all
#### Keep positions of the compliment of the kmer in same list(of 2 lists)
#### with dataframe to store results

In [8]:
index = seqs.keys()
kmers = kmers[:20]# only use 20 of them
kmer_pos =  pd.DataFrame(index=index, columns=kmers)

In [9]:
%%time
for strain,s in seqs.items(): 
    seq = s.upper()
    for kmer in kmers:
        k_p =[]
        #print(strain,kmer)
        k_p.append (list(find_all(seq,kmer)))
        k_p.append (list(find_all(seq,compliment(kmer))))
        kmer_pos.at[strain,kmer] = k_p
 

CPU times: user 2min 46s, sys: 826 ms, total: 2min 47s
Wall time: 2min 47s


In [10]:
kmer_pos.head()

kmer,AAAAAGGGGCTTGCGGAACAGTGTTTATCTT,AAAAGGATGGCTTCGTCATTCCCGCGCAGGC,AAAAGGGGCTTGCGGAACAGTGTTTATCTTT,AAACACTGTTCCGCAAGCCCCTTTTTCAGAC,AAAGGATGGCTTCGTCATTCCCGCGCAGGCG,AAAGGGGCTTGCGGAACAGTGTTTATCTTTG,AACAAAGATAAACACTGTTCCGCAAGCCCCT,AACACTGTTCCGCAAGCCCCTTTTTCAGACC,AACCCGTACCGGTTTTTGTTAATCCGCTATA,AAGCCCCTTTTTCAGACCGCTGGCTAAAAGA,AAGGATGGCTTCGTCATTCCCGCGCAGGCGG,AAGGGGCTTGCGGAACAGTGTTTATCTTTGT,AATGGCGGGTTTTAGGATTACGGTGTATCGG,AATTTACCGGAAAGGAAAAAAATGAAGAAGC,ACACCGTAATCCTAAAACCCGCCATTCCCGC,ACACTGTTCCGCAAGCCCCTTTTTCAGACCG,ACCGTAATCCTAAAACCCGCCATTCCCGCGC,ACTGTTCCGCAAGCCCCTTTTTCAGACCGCT,ACTTCGGCACACCGCCCCGGCAGCTAAAAAT,AGAGAGAGGGCAACAAGCCGCAAGGCTTGTA
485.340,"[[], []]","[[2175075, 2177054, 2177605], []]","[[], []]","[[], []]","[[2175076, 2177055, 2177606], []]","[[], []]","[[], []]","[[], []]","[[2108003], [601665, 1268208]]","[[], []]","[[2175077, 2177056, 2177607], []]","[[], []]","[[2145058], [1866519]]","[[], [2163504, 2200047]]","[[1866524], [2145053]]","[[], []]","[[1866526], [2145051]]","[[], []]","[[1544824], [210510, 1359235]]","[[1008483, 2168628, 2168909], []]"
485.450,"[[1599061, 2036868, 2108366], [2062266]]","[[], [1543116]]","[[1599062, 2036869, 2108367], [2062265]]","[[2062271], [1599056, 2036863, 2108361]]","[[], [1543115]]","[[1599063, 2036870, 2108368], [2062264]]","[[2062262], [1599065, 2036872, 2108370]]","[[2062272], [1599055, 2036862, 2108360]]","[[1926331, 2080794], []]","[[2062285], [1599042, 2036849, 2108347]]","[[], [1543114]]","[[1599064, 2036871, 2108369], [2062263]]","[[], [1221922]]","[[2046638], []]","[[1221927], []]","[[2062273], [1599054, 2036861, 2108359]]","[[1221929], []]","[[2062275], [1599052, 2036859, 2108357]]","[[525878], [1421311]]","[[], [1834137, 1834698, 1834979]]"
485.728,"[[], []]","[[771682, 2018240], []]","[[], []]","[[], []]","[[771683, 2018241], []]","[[], []]","[[], []]","[[], []]","[[], [1552469, 2046373]]","[[], []]","[[771684, 2018242], []]","[[], []]","[[1739566], [1440822]]","[[], [2070391, 2093310]]","[[1440827], [1739561]]","[[], []]","[[1440829], [1739559]]","[[], []]","[[608866, 2075531], []]","[[], [2126387, 2126668]]"
485.787,"[[1681092, 2088416], []]","[[571040], [1929460]]","[[1681093, 2088417], []]","[[], [1681087, 2088411]]","[[571041], [1929459]]","[[1681094, 2088418], []]","[[], [1681096, 2088420]]","[[], [1681086, 2088410]]","[[], [642221]]","[[], [1681073, 2088397]]","[[571042], [1929458]]","[[1681095, 2088419], []]","[[1743540, 2076684], []]","[[], [2113611]]","[[], [1743535, 2076679]]","[[], [1681085, 2088409]]","[[], [1743533, 2076677]]","[[], [1681083, 2088407]]","[[], [172069, 1983786]]","[[], [1984391, 2148887, 2149448]]"
485.346,"[[], []]","[[3909273], [966083]]","[[], []]","[[], []]","[[3909274], [966082]]","[[], []]","[[], []]","[[], []]","[[1326893, 2306666], []]","[[], []]","[[3909275], [966081]]","[[], []]","[[2125901], [4026330]]","[[4115887, 4136497], [4126450]]","[[4026335], [2125896]]","[[], []]","[[4026337], [2125894]]","[[], []]","[[514, 4042348], []]","[[1322066, 1322347], []]"


### store results in a dicionary

In [11]:
%%time
strain_kmer_pos = defaultdict(list)
for strain,s in seqs.items(): 
    seq = s.upper()
    kmer_pos = []
    for kmer in kmers:
        kmer_pos.append(list(find_all(seq,kmer)))
        kmer_pos.append(list(find_all(seq,compliment(kmer))))     
    strain_kmer_pos[strain]= kmer_pos

CPU times: user 2min 46s, sys: 332 ms, total: 2min 46s
Wall time: 2min 46s


In [ ]:
count = 0
for s,d in strain_kmer_pos.items():
    for k,v in d.items():
        if len(v) > 0:
            count += l
            print(s,k, v)
print (count)

## Same Using REGEX
Need this when using Motifs instead of kmers.  
First example as above dataframes.

In [12]:
#%%time
index = seqs.keys()
km = kmers[:20]# only use 10 of them
kmer_pos =  pd.DataFrame(index=index, columns=km)


In [13]:
%%time
for strain,s in seqs.items(): 
    seq = s.upper()
    for kmer in km:
        k_p =[]
        #print(strain,kmer)
        k_p.append ([m.start() for m in re.finditer(kmer, seq)])
        k_p.append ([m.start() for m in re.finditer(compliment(kmer), seq)])
        kmer_pos.at[strain,kmer] = k_p
 

CPU times: user 2min 1s, sys: 384 ms, total: 2min 1s
Wall time: 2min 1s


### Alternative
Using a dict of dicts - outer dict key = strain,  inner dict key = kmer

In [15]:
%%time
strain_kmer_pos = defaultdict(list)

for i,(strain,seq) in enumerate(seqs.items()): 
    s = seq.upper()
    kmer_pos = defaultdict(list)
    for kmer in kmers:
        rev_kmer = compliment(kmer.upper())
        kmer_pos[kmer] = ([m.start() for m in re.finditer(kmer, s)])
        # storing positions of kmer and it's compliment in same list
        #kmer_pos[kmer].extend([m.start() for m in re.finditer(rev_kmer, seq)])
        # alternative - so know that it's on reverse strand
        kmer_pos[f'rev_{kmer}']=([m.start() for m in re.finditer(rev_kmer, s)])  
        
    strain_kmer_pos[strain]= kmer_pos

CPU times: user 2min 1s, sys: 300 ms, total: 2min 2s
Wall time: 2min 2s


In [ ]:
for s,d in strain_kmer_pos.items():
    for k,v in d.items():
        if len(v) > 3:
            print(s,k, v)
#AAAAAGGGGCTTGCGGAACAGTGTTTATCTT 

### Exploring the feature matrix

In [ ]:
features_df = pd.read_pickle('features.pkl')
X =features_df.to_numpy()

In [ ]:
print('sparsity = ',1.0 - np.count_nonzero(X) / X.size)
Z =np.sum(X, axis=0)
print ('column sparsity=', 1.0 - np.count_nonzero(Z) / len(Z))

#### How often so the kmers occur at least once in a sequence

In [ ]:
Z2 = np.count_nonzero(X, axis=0)
len(Z2)  # All kmers occur in atleast one sequence

In [ ]:
import seaborn as sns
data = (Z2) #X.flatten()
fig, ax =plt.subplots(1,2)
sns.distplot(data,bins=np.arange(5, data.max()+1),kde=False,hist_kws={"align" : "left"},  ax=ax[1])
sns.distplot(data,bins=np.arange(0, 30),kde=False,hist_kws={"align" : "left"},ax=ax[0])
fig.show()

In [ ]:
len([z for z in Z2 if z >10 and z <300])